In [157]:
from sklearn.metrics import confusion_matrix, precision_score, accuracy_score, recall_score, f1_score, fbeta_score
import pandas as pd
import numpy as np
from tqdm import tqdm

In [158]:
#Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [159]:
#Read csv data
prometeus = pd.read_csv("prometeus_all_points.csv")
conagua = pd.read_csv("conagua_all_points.csv")
gpm_ = pd.read_csv("gpm_all_points.csv")

In [160]:
pro = prometeus["prometeus"]
con = conagua["conagua"]
gpm = gpm_["gpm"]

In [162]:
score = pd.DataFrame()
from joblib import parallel_backend

with parallel_backend('threading', n_jobs=10):
    
    for tolerance in tqdm(np.arange(0, 5, 0.1)): 
        tpro = pro
        tcon = con
        tgpm = gpm

        dfpro = tpro.apply(lambda x: 1 if x > tolerance else 0)
        dfcon = tcon.apply(lambda x: 1 if x > tolerance else 0)
        dfgpm = tgpm.apply(lambda x: 1 if x > tolerance else 0)

        tnp, fpp, fnp, tpp = confusion_matrix(dfgpm, dfpro).ravel()
        tnc, fpc, fnc, tpc = confusion_matrix(dfgpm, dfcon).ravel()

        list = {
            "Tol": [tolerance.round(2)],
            "AC_pro": [accuracy_score(dfgpm, dfpro).round(3)],
            "PR_pro": [precision_score(dfgpm, dfpro).round(3)],
            "RE_pro": [recall_score(dfgpm, dfpro).round(3)],
            "%_e_pro": [((dfpro[dfpro == 1].count()*100)/dfpro.count()).round(3)],
            "TN_pro": [tnp],
            "FP_pro": [fpp],
            "FN_pro": [fnp],
            "TP_pro": [tpp],
            "AC_con": [accuracy_score(dfgpm, dfcon).round(3)],        
            "PR_con": [precision_score(dfgpm, dfcon).round(3)],
            "RE_con": [recall_score(dfgpm, dfcon).round(3)],
            "%_e_con": [((dfcon[dfcon == 1].count()*100)/dfcon.count()).round(3)],
            "TN_con": [tnc],
            "FP_con": [fpc],
            "FN_con": [fnc],
            "TP_con": [tpc],
            "%_e_gpm": [((dfgpm[dfgpm == 1].count()*100)/dfgpm.count()).round(3)]        
        }

        score = score.append(pd.DataFrame.from_dict(list, orient='columns'))


100%|██████████| 50/50 [35:40<00:00, 42.81s/it]


### Información de las columnas  

Tol: Tolerancia 0,0.1, 0.2, ... 4.9  
%\_e\_gpm: Porcentaje de eventos de precipitacion con esta tolerancia para GPM

#### Prometeus 
AC_pro: Accuracy  
PR_pro: Precision  
RE_pro: Recall  
%\_e\_pro: Porcentaje de eventos de precipitacion con esta tolerancia  
TN_pro: True Negatives  
FP_pro: False Positivess  
FN_pro: False Negatives  
TP_pro: True Positives  

#### Conagua
AC_con: Accuracy  
PR_con: Precision  
RE_con: Recall  
%\_e\_con: Porcentaje de eventos de precipitacion con esta tolerancia  
TN_con: True Negatives  
FP_con: False Positives  
FN_con: False Negatives  
TP_con: True Positives  

In [163]:
score

,Tol,AC_pro,PR_pro,RE_pro,%_e_pro,TN_pro,FP_pro,FN_pro,TP_pro,AC_con,PR_con,RE_con,%_e_con,TN_con,FP_con,FN_con,TP_con,%_e_gpm
0,0.0,0.555,0.508,0.249,22.009,5857286,1432894,4452484,1476536,0.540,0.486,0.445,41.052,4501782,2788398,3290694,2638326,44.852
0,0.1,0.687,0.339,0.193,15.041,8411575,1314789,2819329,673507,0.614,0.303,0.354,30.861,6881690,2844674,2257921,1234915,26.422
0,0.2,0.723,0.300,0.176,13.156,9042656,1217925,2437401,521218,0.647,0.262,0.318,27.187,7608371,2652210,2016915,941704,22.381
0,0.3,0.747,0.276,0.165,11.926,9436802,1140831,2205891,435676,0.672,0.239,0.294,24.577,8104252,2473381,1866048,775519,19.983
0,0.4,0.765,0.259,0.157,11.016,9729272,1078642,2033655,377631,0.692,0.222,0.274,22.514,8492815,2315099,1750199,661087,18.241
0,0.5,0.779,0.246,0.150,10.290,9964795,1025490,1894178,334737,0.710,0.209,0.258,20.803,8815637,2174648,1653549,575366,16.861
0,0.6,0.792,0.235,0.145,9.686,10163053,979755,1775748,300644,0.726,0.198,0.244,19.321,9095358,2047450,1569793,506599,15.707
0,0.7,0.802,0.225,0.140,9.170,10334502,939224,1672562,272912,0.741,0.189,0.231,18.000,9344462,1929264,1495293,450181,14.717
0,0.8,0.812,0.217,0.136,8.723,10483596,903305,1582455,249844,0.754,0.181,0.220,16.808,9568047,1818854,1429223,403076,13.861
0,0.9,0.820,0.209,0.133,8.328,10614469,870451,1503894,230386,0.767,0.175,0.209,15.723,9769707,1715213,1371101,363179,13.119


In [164]:
score.to_csv("score.csv", index=False)

In [165]:
from tabulate import tabulate

print(tabulate(score, headers = 'keys', tablefmt = 'psql'))

+----+-------+----------+----------+----------+-----------+-------------+------------------+------------------+------------------+----------+----------+----------+-----------+-------------+------------------+------------------+------------------+-----------+
|    |   Tol |   AC_pro |   PR_pro |   RE_pro |   %_e_pro |      TN_pro |           FP_pro |           FN_pro |           TP_pro |   AC_con |   PR_con |   RE_con |   %_e_con |      TN_con |           FP_con |           FN_con |           TP_con |   %_e_gpm |
|----+-------+----------+----------+----------+-----------+-------------+------------------+------------------+------------------+----------+----------+----------+-----------+-------------+------------------+------------------+------------------+-----------|
|  0 |   0   |    0.555 |    0.508 |    0.249 |    22.009 | 5.85729e+06 |      1.43289e+06 |      4.45248e+06 |      1.47654e+06 |    0.54  |    0.486 |    0.445 |    41.052 | 4.50178e+06 |      2.7884e+06  |      3.29069e+